In [1]:
import pandas as pd

In [2]:
predictions = pd.read_csv("../data/07_model_output/ml_technique_modeling/predictions_merged")
performance = pd.read_csv(
    "../data/07_model_output/ml_technique_modeling/performance_concatenated"
)

In [4]:
performance

,Unnamed: 0,Fold,MSE,MAPE,MAE,R2,RMSE,Ticker,RMSLE
0,0,Mean,6.5897,0.0259,1.8701,0.8670,2.4682,NKE,0.0329
1,1,Mean,69.9783,0.0323,6.4898,0.8504,7.9100,GS,0.0391


In [7]:
pred_cols = ["GS_prediction_label", "NKE_prediction_label"]
stocks = predictions.loc[:, pred_cols]

In [9]:
returns = stocks.pct_change().dropna()

In [11]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize


def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate=0.0):
    # Annualize returns and volatility
    returns = np.sum(mean_returns * weights) * 252  # Annualizing the return
    std = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(
        252
    )  # Annualizing the volatility
    sharpe_ratio = (returns - risk_free_rate) / std  # Calculating the Sharpe Ratio
    return returns, std, sharpe_ratio


def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.0):
    _, _, sharpe_ratio = portfolio_performance(
        weights, mean_returns, cov_matrix, risk_free_rate
    )
    return -sharpe_ratio


def get_constraints():
    return {"type": "eq", "fun": lambda x: np.sum(x) - 1}


def get_bounds(num_assets):
    return tuple((0, 1) for asset in range(num_assets))


num_assets = len(returns.columns)
initial_guess = num_assets * [
    1.0 / num_assets,
]

mean_returns = returns.mean()
cov_matrix = returns.cov()
risk_free_rate = 0.0  # Adjust as needed

result = minimize(
    negative_sharpe_ratio,
    initial_guess,
    args=(mean_returns, cov_matrix, risk_free_rate),
    method="SLSQP",
    bounds=get_bounds(num_assets),
    constraints=get_constraints(),
)

optimal_weights = result.x

portfolio_return, portfolio_volatility, portfolio_sharpe = portfolio_performance(
    optimal_weights, mean_returns, cov_matrix, risk_free_rate
)

print("Optimal Weights: ", optimal_weights)
print("Expected Portfolio Return: ", portfolio_return)
print("Expected Portfolio Volatility: ", portfolio_volatility)
print("Sharpe Ratio: ", portfolio_sharpe)

Optimal Weights:  [0.40223445 0.59776555]
Expected Portfolio Return:  0.13563560654558243
Expected Portfolio Volatility:  0.22708966719750132
Sharpe Ratio:  0.5972777547276922
